В этом блоке мы продемонстрировали дообучение модели Hubert на наборе данных marsyas/gtzan для классификации музыки. Accuracy нашего примера составила 83%. Ваша задача - улучшить этот показатель.

Вы можете выбрать любую модель на 🤗 Hugging Face Hub, которая, по вашему мнению, подходит для классификации аудио, и использовать точно такой же набор данных marsyas/gtzan для построения собственного классификатора.

Ваша цель - достичь accuracy 87% на этом наборе данных с помощью вашего классификатора. Вы можете выбрать точно такую же модель, поиграть с гиперпараметрами обучения или выбрать совершенно другую модель - все зависит от вас!

Чтобы ваш результат был засчитан в сертификат, не забудьте в конце обучения вывести модель на Hub, как это было показано в данном блоке, со следующими **kwargs:

In [1]:
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.8 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
! pip install datasets
!pip install git+https://github.com/huggingface/transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mhhwqwy3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-mhhwqwy3
  Resolved https://github.com/huggingface/transformers to commit 0a365c3e6a0e174302debff4023182838607acf1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.33.0.de

In [4]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.3 MB/s eta 0:00:00


In [5]:
import numpy as np
from datasets import load_dataset
from datasets import Audio
from transformers import AutoFeatureExtractor
from transformers import AutoModelForAudioClassification
from transformers import TrainingArguments
from transformers import Trainer
from collections import Counter
from sklearn.model_selection import train_test_split
import evaluate


Необходимо дообучить модель (модель выбрать на свое усмотрение) для классификации музыкальных жанров на датасете [marsyas/gtzan](https://huggingface.co/datasets/marsyas/gtzan)

GTZAN представляет собой популярный набор данных из 1000 песен для классификации музыкальных жанров. Каждая песня представляет собой 30-секундный клип из одного из 10 музыкальных жанров - от диско до металла.

In [6]:
# импортируем датасет
gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

In [7]:
gtzan.column_names

{'train': ['file', 'audio', 'genre']}

In [8]:
gtzan.data

{'train': MemoryMappedTable
 file: string
 audio: struct<bytes: binary, path: string>
   child 0, bytes: binary
   child 1, path: string
 genre: int64
 ----
 file: [["/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/blues/blues.00000.wav","/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/blues/blues.00001.wav","/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/blues/blues.00002.wav","/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/blues/blues.00003.wav","/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/blues/blues.00004.wav",...,"/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c605

Посмотрим на баланс классов. По выводу ячейки ниже можно понять, что датасет сбалансирован по всем классам.

In [9]:
с = Counter(gtzan['train']['genre'])
с

Counter({0: 100,
         1: 100,
         2: 100,
         3: 100,
         4: 100,
         5: 99,
         6: 100,
         7: 100,
         8: 100,
         9: 100})

Так как датасет изначально не был разбит на обучающую и тренировочную выборки, сделаем это с помощью *train_test_split()* из sklearn

In [10]:
data = gtzan['train'].train_test_split(seed=42, shuffle=True, test_size=0.1)
data

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [11]:
# первый элемент тренировочной выборки
data['train'][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229]),
  'sampling_rate': 22050},
 'genre': 7}

In [12]:
id2label_fn = data["train"].features["genre"].int2str # list ClassLabel names
id2label_fn(data["train"][0]["genre"])

'pop'

Вызовем *feature extractor* для выбранной модели.

In [13]:
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [14]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [15]:
data = data.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [16]:
data["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

In [17]:
sample = data["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


In [18]:
sample["sampling_rate"]

16000

In [19]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -7.45e-09, Variance: 1.0


In [20]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [21]:
data_encoded = data.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [22]:
data_encoded

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

In [23]:
data_encoded.column_names

{'train': ['genre', 'input_values', 'attention_mask'],
 'test': ['genre', 'input_values', 'attention_mask']}

In [24]:
data_encoded = data_encoded.rename_column("genre", "label")

In [25]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(data_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

In [26]:
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.weight', 'projector.bias', 'projector.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
model_name = model_id.split("/")[-1]
batch_size = 4
gradient_accumulation_steps = 2
num_train_epochs = 10

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    adam_beta1 = 0.9,
    adam_beta2 = 0.999,
    adam_epsilon = 1e-08,
    lr_scheduler_type = 'linear',
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
)

In [31]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [34]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=data_encoded["train"],
    eval_dataset=data_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,1.996400,1.844229,0.500000
2,1.350100,1.265964,0.680000
2,0.921000,0.929255,0.750000
4,0.655000,0.913501,0.730000
4,0.391100,0.685544,0.760000
6,0.228600,0.595057,0.820000
6,0.209500,0.558273,0.820000
8,0.179800,0.462578,0.870000
8,0.257400,0.504303,0.850000
9,0.179600,0.479410,0.870000


TrainOutput(global_step=1120, training_loss=0.7517030929880483, metrics={'train_runtime': 6328.3139, 'train_samples_per_second': 1.421, 'train_steps_per_second': 0.177, 'total_flos': 6.1073780918976e+17, 'train_loss': 0.7517030929880483, 'epoch': 9.96})

In [ ]:
#model = AutoModelForAudioClassification.from_pretrained("/content/distilhubert-finetuned-gtzan/checkpoint-900")

In [35]:
tokenizer=feature_extractor

In [36]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

trainer.push_to_hub(**kwargs)
tokenizer.push_to_hub(f"{model_name}-finetuned-gtzan")

CommitInfo(commit_url='https://huggingface.co/Lightmourne/distilhubert-finetuned-gtzan/commit/c9d5eed4064e0b450f5be75b61d723867271256c', commit_message='Upload feature extractor', commit_description='', oid='c9d5eed4064e0b450f5be75b61d723867271256c', pr_url=None, pr_revision=None, pr_num=None)